# Quackling — Native Nodes

In [1]:
# requirements for this example:
%pip install -qq \
    quackling \
    python-dotenv \
    llama-index-embeddings-huggingface \
    llama-index-llms-huggingface-api \
    llama-index-vector-stores-milvus \
    jsonpath-ng

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from tempfile import TemporaryDirectory

from dotenv import load_dotenv
from pydantic import TypeAdapter
from rich.pretty import pprint

load_dotenv()

FILE_PATHS = [
    # "/path/to/local/pdf",  # file path
    "https://arxiv.org/pdf/2206.01062",  # URL (DocLayNet paper)
]
TEXT_QA_TEMPLATE_STR = "Context information is below.\n---------------------\n{context_str}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: {query_str}\nAnswer:\n"
QUERY = "How many pages were human annotated?"
TOP_K = 3

INGEST = TypeAdapter(bool).validate_python(os.environ.get("INGEST", "True"))

In [3]:
import warnings

warnings.filterwarnings(action="ignore", category=UserWarning, module="pydantic|torch")
warnings.filterwarnings(action="ignore", category=FutureWarning, module="easyocr")

## Setup

### Reader and node parser

Below we set up:
- a `DoclingPDFReader` which will be used to create LlamaIndex documents, and
- a `NodeParser`, which will be used to create LlamaIndex nodes out of the documents

To leverage Docling's rich document structure format, we namely set the parse type to JSON and use a `HierarchicalJSONNodeParser` accordingly:

In [4]:
from quackling.llama_index.node_parsers import HierarchicalJSONNodeParser
from quackling.llama_index.readers import DoclingPDFReader

reader = DoclingPDFReader(parse_type=DoclingPDFReader.ParseType.JSON)
node_parser = HierarchicalJSONNodeParser()

### Embed model

In [5]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

HF_EMBED_MODEL_ID = "BAAI/bge-small-en-v1.5"

embed_model = HuggingFaceEmbedding(model_name=HF_EMBED_MODEL_ID)

### LLM

In [6]:
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI

HF_API_KEY = os.environ.get("HF_API_KEY")
HF_LLM_MODEL_ID = "mistralai/Mixtral-8x7B-Instruct-v0.1"

llm = HuggingFaceInferenceAPI(
    token=HF_API_KEY,
    model_name=HF_LLM_MODEL_ID,
)

### Vector store

In [7]:
from llama_index.vector_stores.milvus import MilvusVectorStore

MILVUS_URL = os.environ.get(
    "MILVUS_URL", f"{(tmp_dir := TemporaryDirectory()).name}/milvus_demo.db"
)
MILVUS_COLL_NAME = os.environ.get("MILVUS_COLL_NAME", "quackling_native_nodes")
MILVUS_KWARGS = TypeAdapter(dict).validate_json(os.environ.get("MILVUS_KWARGS", "{}"))

vector_store = MilvusVectorStore(
    uri=MILVUS_URL,
    collection_name=MILVUS_COLL_NAME,
    dim=len(embed_model.get_text_embedding("hi")),
    overwrite=INGEST,  # not showing follow-up/incremental ingestions in this example
    **MILVUS_KWARGS,
)

In [8]:
from llama_index.core.vector_stores.types import VectorStoreQueryMode

vector_store_query_mode = VectorStoreQueryMode.DEFAULT  # i.e. dense

## Preparing vector store index

In [9]:
from llama_index.core import StorageContext, VectorStoreIndex

if INGEST:
    # in this case we ingest the data into the vector store
    docs = reader.load_data(file_path=FILE_PATHS)
    pprint(docs, max_length=1, max_string=50, max_depth=4)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    index = VectorStoreIndex.from_documents(
        documents=docs,
        embed_model=embed_model,
        storage_context=storage_context,
        transformations=[
            node_parser,
        ],
    )
else:
    # in this case we just load the vector store index
    index = VectorStoreIndex.from_vector_store(vector_store)

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

[
│   Document(
│   │   id_='5dfbd8c115a15fd3396b68409124cfee29fc8efac7b5c84663'+14,
│   │   embedding=None,
│   │   metadata={'dl_doc_hash': '5dfbd8c115a15fd3396b68409124cfee29fc8efac7b5c84663'+14},
│   │   excluded_embed_metadata_keys=['dl_doc_hash'],
│   │   excluded_llm_metadata_keys=['dl_doc_hash'],
│   │   relationships={},
│   │   text='{"_name":"","type":"pdf-document","description":{"'+171405,
│   │   mimetype='text/plain',
│   │   start_char_idx=None,
│   │   end_char_idx=None,
│   │   text_template='{metadata_str}\n\n{content}',
│   │   metadata_template='{key}: {value}',
│   │   metadata_seperator='\n'
│   )
]

## RAG

In [10]:
from llama_index.core import PromptTemplate
from llama_index.core.response_synthesizers.type import ResponseMode

query_engine = index.as_query_engine(
    llm=llm,
    similarity_top_k=TOP_K,
    text_qa_template=PromptTemplate(TEXT_QA_TEMPLATE_STR),
    vector_store_query_mode=vector_store_query_mode,
    response_mode=ResponseMode.REFINE,
)
query_res = query_engine.query(QUERY)
pprint(query_res, max_length=1, max_string=250, max_depth=4)

Response(
│   response='80,000 pages were human annotated, with a total of 91,104 annotation instances.',
│   source_nodes=[
│   │   NodeWithScore(
│   │   │   node=TextNode(
│   │   │   │   id_='578c1d76-ea41-4b03-949e-dbd3f0a71694',
│   │   │   │   embedding=None,
│   │   │   │   metadata={...},
│   │   │   │   excluded_embed_metadata_keys=[...],
│   │   │   │   excluded_llm_metadata_keys=[...],
│   │   │   │   relationships={...},
│   │   │   │   text='4 ANNOTATION CAMPAIGN\nThe complete annotation guideline is over 100 pages long and a detailed description is obviously out of scope for this paper. Nevertheless, it will be made publicly available alongside with DocLayNet for future reference.',
│   │   │   │   mimetype='text/plain',
│   │   │   │   start_char_idx=None,
│   │   │   │   end_char_idx=None,
│   │   │   │   text_template='{metadata_str}\n\n{content}',
│   │   │   │   metadata_template='{key}: {value}',
│   │   │   │   metadata_seperator='\n'
│   │   │   ),
│   │   │   score=0.7986579537391663
│   │   ),
│   │   ... +2
│   ],
│   metadata={'578c1d76-ea41-4b03-949e-dbd3f0a71694': {'path': '$.main-text[74]'}, ... +2}
)

## Retrieval

We can also just run the retrieval without generation:

In [11]:
retriever = index.as_retriever(
    similarity_top_k=TOP_K,
    vector_store_query_mode=vector_store_query_mode,
)
retr_res = retriever.retrieve(QUERY)
pprint(retr_res, max_length=1, max_string=250, max_depth=4)

[
│   NodeWithScore(
│   │   node=TextNode(
│   │   │   id_='578c1d76-ea41-4b03-949e-dbd3f0a71694',
│   │   │   embedding=None,
│   │   │   metadata={'path': '$.main-text[74]'},
│   │   │   excluded_embed_metadata_keys=['path'],
│   │   │   excluded_llm_metadata_keys=['path'],
│   │   │   relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(...), ... +2},
│   │   │   text='4 ANNOTATION CAMPAIGN\nThe complete annotation guideline is over 100 pages long and a detailed description is obviously out of scope for this paper. Nevertheless, it will be made publicly available alongside with DocLayNet for future reference.',
│   │   │   mimetype='text/plain',
│   │   │   start_char_idx=None,
│   │   │   end_char_idx=None,
│   │   │   text_template='{metadata_str}\n\n{content}',
│   │   │   metadata_template='{key}: {value}',
│   │   │   metadata_seperator='\n'
│   │   ),
│   │   score=0.7986579537391663
│   ),
│   ... +2
]

## Fetching Docling native document nodes

In [12]:
import jsonpath_ng
from docling_core.types import BaseText, Document, Ref

NativeNode = BaseText | Ref


def get_native_node(dl_doc: Document, path: str) -> NativeNode:
    jsonpath_expr = jsonpath_ng.parse(path)
    jsonpath_res = [match.value for match in jsonpath_expr.find(dl_doc.model_dump())]
    if (num_res := len(jsonpath_res)) == 0:
        raise RuntimeError(f"No results found for {path}")
    elif num_res > 1:
        # currently only single result supported
        raise RuntimeError(f"Multiple results found for {path}")
    jres = jsonpath_res[0]
    return TypeAdapter(NativeNode).validate_python(jres)

Below we demonstrate fetching the native document nodes based on the JSONPath returned in the retrieval results.

This brings to light rich metadata for each result, including its page and bounding box information.

In [13]:
from docling_core.types import Document
from rich.console import Console

from quackling.llama_index.node_parsers.base import NodeMetadata
from quackling.llama_index.readers.base import DocumentMetadata

console = Console()

dl_docs = [Document.model_validate_json(doc.text) for doc in docs]
for retr_item in retr_res:
    source_metadata = DocumentMetadata.model_validate(
        retr_item.node.source_node.metadata
    )
    dl_doc_hash = source_metadata.dl_doc_hash
    dl_doc = [d for d in dl_docs if d.file_info.document_hash == dl_doc_hash][0]
    path = NodeMetadata.model_validate(retr_item.node.metadata).path
    native_node = get_native_node(dl_doc=dl_doc, path=path)
    console.rule(f"dl_doc_hash={dl_doc_hash[:7]}...\n{path=}")
    pprint(native_node)

────────────────────────────────── dl_doc_hash=5dfbd8c... path='$.main-text[74]' ──────────────────────────────────

BaseText(
│   text='The complete annotation guideline is over 100 pages long and a detailed description is obviously out of scope for this paper. Nevertheless, it will be made publicly available alongside with DocLayNet for future reference.',
│   obj_type='paragraph',
│   name='Text',
│   font=None,
│   prov=[
│   │   Prov(
│   │   │   bbox=[52.994422912597656, 217.798828125, 295.5625305175781, 259.6097106933594],
│   │   │   page=5,
│   │   │   span=[0, 221],
│   │   │   ref_s3_data=None
│   │   )
│   ]
)

────────────────────────────────── dl_doc_hash=5dfbd8c... path='$.main-text[79]' ──────────────────────────────────

BaseText(
│   text="Phase 4: Production annotation. The previously selected 80K pages were annotated with the defined 11 class labels by 32 annotators. This production phase took around three months to complete. All annotations were created online through CCS, which visualises the programmatic PDF text-cells as an overlay on the page. The page annotation are obtained by drawing rectangular bounding-boxes, as shown in Figure 3. With regard to the annotation practices, we implemented a few constraints and capabilities on the tooling level. First, we only allow non-overlapping, vertically oriented, rectangular boxes. For the large majority of documents, this constraint was sufficient and it speeds up the annotation considerably in comparison with arbitrary segmentation shapes. Second, annotator staff were not able to see each other's annotations. This was enforced by design to avoid any bias in the annotation, which could skew the numbers of the inter-annotator agreement (see Table 1). We wanted",
│   obj_type='paragraph',
│   name='Text',
│   font=None,
│   prov=[
│   │   Prov(
│   │   │   bbox=[317.00592041015625, 82.7375717163086, 559.7149047851562, 245.28392028808594],
│   │   │   page=5,
│   │   │   span=[0, 987],
│   │   │   ref_s3_data=None
│   │   )
│   ]
)

────────────────────────────────── dl_doc_hash=5dfbd8c... path='$.main-text[36]' ──────────────────────────────────

BaseText(
│   text='DocLayNet contains 80863 PDF pages. Among these, 7059 carry two instances of human annotations, and 1591 carry three. This amounts to 91104 total annotation instances. The annotations provide layout information in the shape of labeled, rectangular boundingboxes. We define 11 distinct labels for layout features, namely Caption , Footnote , Formula , List-item , Page-footer , Page-header , Picture , Section-header , Table , Text , and Title . Our reasoning for picking this particular label set is detailed in Section 4.',
│   obj_type='paragraph',
│   name='Text',
│   font=None,
│   prov=[
│   │   Prov(
│   │   │   bbox=[317.11236572265625, 116.19312286376953, 559.7131958007812, 202.27523803710938],
│   │   │   page=2,
│   │   │   span=[0, 522],
│   │   │   ref_s3_data=None
│   │   )
│   ]
)